In [ ]:
'''
-Each entity should be one to one with each table
-No redundant code (business logic)
-No redundant data (data model / schema)
-Uniform interface between entities
-Business logic validation for inserting new data and updating data

'''





In [20]:
import pymysql
import yaml
from pathlib import Path




class baseObject:
    def setup(self,tn):
        self.tn = tn
        self.conn = None
        self.cur = None
        self.pk = None
        self.fields = []
        self.errors = []
        self.data = []
        self.establishConnection()
        self.getFields()
    def establishConnection(self):
        config = yaml.safe_load(Path("config.yml").read_text())
        #print(config)
        self.conn = pymysql.connect(host=config['db']['host'], port=config['db']['port'], user=config['db']['user'],
                       passwd=config['db']['passwd'], db=config['db']['db'], autocommit=True)
        self.cur = self.conn.cursor(pymysql.cursors.DictCursor) 
    def set(self,d):
        self.data.append(d)
    def getFields(self):
        sql = f'''DESCRIBE `{self.tn}`;'''
        self.cur.execute(sql)
        for row in self.cur:
            if row['Extra'] == 'auto_increment':
                self.pk  = row['Field']
            else:
                self.fields.append(row['Field'])
    def insert(self,n=0):
        count = 0
        vals = []
        sql = f"INSERT INTO `{self.tn}` ("
        for field in self.fields:
            sql += f"`{field}`,"
            vals.append(self.data[n][field])
            count +=1
        sql = sql[0:-1] + ') VALUES ('
        tokens = ("%s," * count)[0:-1]
        sql += tokens + ');'
        #print(sql,vals)
        self.cur.execute(sql,vals)
        self.data[n][self.pk] = self.cur.lastrowid
    def getById(self,id):
        sql = f"Select * from `{self.tn}` where `{self.pk}` = %s" 
        #print(sql,id)
        self.cur.execute(sql,(id))
        self.data = []
        for row in self.cur:
            self.data.append(row)
    def getAll(self):
        sql = f"Select * from `{self.tn}`" 
        self.cur.execute(sql)
        self.data = []
        for row in self.cur:
            self.data.append(row)
    def truncate(self):
        sql = f"TRUNCATE TABLE `{self.tn}`" 
        self.cur.execute(sql)
    def getByField(self,field,val):
        sql = f"Select * from `{self.tn}` where `{field}` = %s" 
        #print(sql,val)
        self.cur.execute(sql,(val))
        self.data = []
        for row in self.cur:
            self.data.append(row)
    def update(self,n=0):
        vals=[]
        fvs=''
        for field in self.fields:
            if field in self.data[n].keys():
                fvs += f"`{field}`=%s,"
                vals.append(self.data[n][field])
        fvs=fvs[:-1]
        sql=f"UPDATE `{self.tn}` SET {fvs} WHERE `{self.pk}` = %s"
        vals.append(self.data[n][self.pk])
        #print(sql,vals)
        self.cur.execute(sql,vals)
    def deleteById(self,id):
        sql = f"Delete from `{self.tn}` where `{self.pk}` = %s" 
        self.cur.execute(sql,(id))

import hashlib

class user(baseObject):
    def __init__(self):
        self.setup('demo_users')
        self.roles = [{'value':'admin','text':'admin'},{'value':'employee','text':'employee'},{'value':'customer','text':'customer'}]
    def hashPassword(self,pw):
        pw = pw+'xyz'
        return hashlib.md5(pw.encode('utf-8')).hexdigest()
    def verify_new(self,n=0):
        self.errors = []

        if self.data[n]['name'] == '':
            self.errors.append('Name cannot be blank.')
        else:
            u = user()
            u.getByField('name',self.data[n]['name'])
            if len(u.data) > 0:
                self.errors.append('Name already in use.')
        if self.data[n]['password'] != self.data[n]['password2']:
            self.errors.append('Retyped password must match.')
        if len(self.data[n]['password']) < 3:
            self.errors.append('Password needs to be more than 3 chars.')
        else:
            self.data[n]['password'] = self.hashPassword(self.data[n]['password'])
        rl = []
        for role in self.roles:
            rl.append(role['value'])
        if self.data[n]['role'] not in rl:
            self.errors.append(f'Role must be one of {rl}')
    
        

        ##Include this in verify:
        if len(self.errors) > 0:
            return False
        else:
            return True
    def verify_update(self,n=0):
        self.errors = []

        if self.data[n]['name'] == '':
            self.errors.append('Name cannot be blank.')
        else:
            u = user()
            u.getByField('name',self.data[n]['name'])
            if len(u.data) > 0 and u.data[0][u.pk] != self.data[n][self.pk]:
                self.errors.append('Name already in use.')
        
        if len(self.data[n]['password']) < 3:
            self.errors.append('Password needs to be more than 3 chars.')
        else:
            self.data[n]['password'] = self.hashPassword(self.data[n]['password'])
        rl = []
        for role in self.roles:
            rl.append(role['value'])
        if self.data[n]['role'] not in rl:
            self.errors.append(f'Role must be one of {rl}')
        if len(self.data[n]['password']) > 0: #user intends to change pw
            if self.data[n]['password'] != self.data[n]['password2']:
                self.errors.append('Retyped password must match.')
            if len(self.data[n]['password']) < 3:
                self.errors.append('Password must be > 4 chars.')
            else:
                self.data[n]['password'] = self.hashPassword(self.data[n]['password'])
        else:
            del self.data[n]['password']
    
        

        ##Include this in verify:
        if len(self.errors) > 0:
            return False
        else:
            return True
u = user()
u.truncate()
u.set({'name':'conlontj','password':'134','password2':'134','role':'admin'})
if u.verify_new():
    print(u.data[0]['password'])
    u.insert()
    print(u.data[0][u.pk])
else:
    print(u.errors)
#u.insert()
#print(u.data[0][u.pk])
'''
u = user()
u.getById(1)
u.data[0]['name'] = 'tconlon'
u.update()
'''
        

09fddf96d18c92c2fd795fd5cbc7dc33
1


"\nu = user()\nu.getById(1)\nu.data[0]['name'] = 'tconlon'\nu.update()\n"